<a href="https://colab.research.google.com/github/ChangHuaHua/QM2-Group-12/blob/main/Chloropleth_Map_Slider.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://dmnfarrell.github.io/bioinformatics/bokeh-maps

**Import libraries and files**

In [7]:
!pip install geopandas
!pip install plotly==4.5

     |████████████████████████████████| 7.1MB 4.5MB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [9]:
import pandas as pd
import geopandas as gpd
import json
import matplotlib as mpl
import pylab as plt

from bokeh.io import output_file, show, output_notebook, export_png
from bokeh.models import ColumnDataSource, GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.plotting import figure
from bokeh.palettes import brewer

import panel as pn
import panel.widgets as pnw

In [3]:
from google.colab import files
uploaded = files.upload()

Saving Africa_FULL_MERGE.csv to Africa_FULL_MERGE.csv
Saving FULL_MERGE.csv to FULL_MERGE.csv


**World Shape File**

In [4]:
from google.colab import files
uploaded = files.upload()

Saving World_Countries__Generalized_.cpg to World_Countries__Generalized_.cpg
Saving World_Countries__Generalized_.dbf to World_Countries__Generalized_.dbf
Saving World_Countries__Generalized_.prj to World_Countries__Generalized_.prj
Saving World_Countries__Generalized_.shp to World_Countries__Generalized_.shp
Saving World_Countries__Generalized_.shx to World_Countries__Generalized_.shx


In [48]:
Data = 'World_Countries__Generalized_.shp'
World_Shape_Data = gpd.read_file(Data)[['COUNTRYAFF', 'AFF_ISO', 'geometry']]
World_Shape_Data['COUNTRYAFF'].replace({"Congo DRC": "Democratic Republic of Congo","Côte d'Ivoire": "Cote d'Ivoire"}, inplace=True)
World_Shape_Data.head()

,COUNTRYAFF,AFF_ISO,geometry
0,United States,US,"POLYGON ((-170.74390 -14.37555, -170.74942 -14..."
1,United States,US,"MULTIPOLYGON (((-160.02114 -0.39805, -160.0281..."
2,New Zealand,NZ,"MULTIPOLYGON (((-159.74698 -21.25667, -159.793..."
3,France,FR,"MULTIPOLYGON (((-149.17920 -17.87084, -149.258..."
4,New Zealand,NZ,"POLYGON ((-169.89389 -19.14556, -169.93088 -19..."


**Merge Africa**

In [13]:
Africa_data_merge = 'Africa_FULL_MERGE.csv'

In [14]:
df = pd.read_csv(Africa_data_merge, encoding = 'latin1')
df[['Country', 'Year', 'Region', 'IncomeGroup', 'Country Code', 'TFR', 'HDI Rank (2018)', 'Females in secondary education (%)', 'Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)']]
df.rename(columns={'Country':'COUNTRY'}, inplace=True)

In [15]:
Africa_Shape_Data = World_Shape_Data.merge(df, left_on = 'COUNTRY', right_on = 'COUNTRY')
Africa_Shape_Data

,FID,COUNTRY,ISO,COUNTRYAFF,AFF_ISO,SHAPE_Leng,SHAPE_Area,geometry,Unnamed: 0,Year,Region,IncomeGroup,Country Code,TFR,HDI Rank (2018),HDI,Females in secondary education (%),"Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)"
0,70,Burkina Faso,BF,Burkina Faso,BF,27.007423,22.634154,"POLYGON ((0.91797 10.99640, 0.81195 10.99694, ...",531,1960,Sub-Saharan Africa,Low income,BFA,6.291,NaN,NaN,NaN,NaN
1,70,Burkina Faso,BF,Burkina Faso,BF,27.007423,22.634154,"POLYGON ((0.91797 10.99640, 0.81195 10.99694, ...",532,1961,Sub-Saharan Africa,Low income,BFA,6.316,NaN,NaN,NaN,NaN
2,70,Burkina Faso,BF,Burkina Faso,BF,27.007423,22.634154,"POLYGON ((0.91797 10.99640, 0.81195 10.99694, ...",533,1962,Sub-Saharan Africa,Low income,BFA,6.344,NaN,NaN,NaN,NaN
3,70,Burkina Faso,BF,Burkina Faso,BF,27.007423,22.634154,"POLYGON ((0.91797 10.99640, 0.81195 10.99694, ...",534,1963,Sub-Saharan Africa,Low income,BFA,6.377,NaN,NaN,NaN,NaN
4,70,Burkina Faso,BF,Burkina Faso,BF,27.007423,22.634154,"POLYGON ((0.91797 10.99640, 0.81195 10.99694, ...",535,1964,Sub-Saharan Africa,Low income,BFA,6.414,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3181,203,Somalia,SO,Somalia,SO,48.995362,52.070402,"POLYGON ((45.86945 10.84416, 45.89750 10.83027...",2709,2014,Sub-Saharan Africa,Low income,SOM,6.462,NaN,NaN,NaN,18.250999
3182,203,Somalia,SO,Somalia,SO,48.995362,52.070402,"POLYGON ((45.86945 10.84416, 45.89750 10.83027...",2710,2015,Sub-Saharan Africa,Low income,SOM,6.363,NaN,NaN,NaN,18.472000
3183,203,Somalia,SO,Somalia,SO,48.995362,52.070402,"POLYGON ((45.86945 10.84416, 45.89750 10.83027...",2711,2016,Sub-Saharan Africa,Low income,SOM,6.265,NaN,NaN,NaN,18.601999
3184,203,Somalia,SO,Somalia,SO,48.995362,52.070402,"POLYGON ((45.86945 10.84416, 45.89750 10.83027...",2712,2017,Sub-Saharan Africa,Low income,SOM,6.168,NaN,NaN,NaN,18.627001


In [16]:
Africa_Shape_Data.to_csv('Africa_Shape_Data.csv')

**Merge World**

In [17]:
World_data_merge = 'FULL_MERGE.csv'
df2 = pd.read_csv(World_data_merge, encoding = 'latin1')
df2[['Country', 'Year', 'Region', 'IncomeGroup', 'Country Code', 'TFR', 'HDI Rank (2018)', 'Females in secondary education (%)', 'Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)']]
df2.rename(columns={'Country':'COUNTRY'}, inplace=True)
World_Shape_Data_Merge = World_Shape_Data.merge(df, left_on = 'COUNTRY', right_on = 'COUNTRY')
World_Shape_Data_Merge

,FID,COUNTRY,ISO,COUNTRYAFF,AFF_ISO,SHAPE_Leng,SHAPE_Area,geometry,Unnamed: 0,Year,Region,IncomeGroup,Country Code,TFR,HDI Rank (2018),HDI,Females in secondary education (%),"Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)"
0,70,Burkina Faso,BF,Burkina Faso,BF,27.007423,22.634154,"POLYGON ((0.91797 10.99640, 0.81195 10.99694, ...",531,1960,Sub-Saharan Africa,Low income,BFA,6.291,NaN,NaN,NaN,NaN
1,70,Burkina Faso,BF,Burkina Faso,BF,27.007423,22.634154,"POLYGON ((0.91797 10.99640, 0.81195 10.99694, ...",532,1961,Sub-Saharan Africa,Low income,BFA,6.316,NaN,NaN,NaN,NaN
2,70,Burkina Faso,BF,Burkina Faso,BF,27.007423,22.634154,"POLYGON ((0.91797 10.99640, 0.81195 10.99694, ...",533,1962,Sub-Saharan Africa,Low income,BFA,6.344,NaN,NaN,NaN,NaN
3,70,Burkina Faso,BF,Burkina Faso,BF,27.007423,22.634154,"POLYGON ((0.91797 10.99640, 0.81195 10.99694, ...",534,1963,Sub-Saharan Africa,Low income,BFA,6.377,NaN,NaN,NaN,NaN
4,70,Burkina Faso,BF,Burkina Faso,BF,27.007423,22.634154,"POLYGON ((0.91797 10.99640, 0.81195 10.99694, ...",535,1964,Sub-Saharan Africa,Low income,BFA,6.414,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3181,203,Somalia,SO,Somalia,SO,48.995362,52.070402,"POLYGON ((45.86945 10.84416, 45.89750 10.83027...",2709,2014,Sub-Saharan Africa,Low income,SOM,6.462,NaN,NaN,NaN,18.250999
3182,203,Somalia,SO,Somalia,SO,48.995362,52.070402,"POLYGON ((45.86945 10.84416, 45.89750 10.83027...",2710,2015,Sub-Saharan Africa,Low income,SOM,6.363,NaN,NaN,NaN,18.472000
3183,203,Somalia,SO,Somalia,SO,48.995362,52.070402,"POLYGON ((45.86945 10.84416, 45.89750 10.83027...",2711,2016,Sub-Saharan Africa,Low income,SOM,6.265,NaN,NaN,NaN,18.601999
3184,203,Somalia,SO,Somalia,SO,48.995362,52.070402,"POLYGON ((45.86945 10.84416, 45.89750 10.83027...",2712,2017,Sub-Saharan Africa,Low income,SOM,6.168,NaN,NaN,NaN,18.627001


In [18]:
World_Shape_Data_Merge.to_csv('World_Shape_Data_Merge.csv')

**Chloropleth Map**

In [24]:
def get_geodatasource(Africa_Shape_Data):    
    """Get getjsondatasource from geopandas object"""
    json_data = json.dumps(json.loads(Africa_Shape_Data.to_json()))
    return GeoJSONDataSource(geojson = json_data)

def bokeh_plot_map(Africa_Shape_Data, column=None, title=''):
    """Plot bokeh map from GeoJSONDataSource """

    geosource = get_geodatasource(Africa_Shape_Data)
    palette = brewer['OrRd'][8]
    palette = palette[::-1]
    vals = World_Shape_Data[column]
    #Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
    color_mapper = LinearColorMapper(palette = palette, low = vals.min(), high = vals.max())
    color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=500, height=20,
                         location=(0,0), orientation='horizontal')

    tools = 'wheel_zoom,pan,reset'
    p = figure(title = title, plot_height=400 , plot_width=850, toolbar_location='right', tools=tools)
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    #Add patch renderer to figure
    p.patches('xs','ys', source=geosource, fill_alpha=1, line_width=0.5, line_color='black',  
              fill_color={'field' :column , 'transform': color_mapper})
    #Specify figure layout.
    p.add_layout(color_bar, 'below')
    return p

**Interactive Slider**

In [53]:
AfricaData = 'Africa_Shape_Data.csv'
get_dataset = pd.read_csv(AfricaData)

def map_dash():
    """Map dashboard"""

    from bokeh.models.widgets import DataTable
    map_pane = pn.pane.Bokeh(width=400)
    data_select = pnw.Select(name='dataset',options=list(Africa_Shape_Data.index))
    year_slider = pnw.IntSlider(start=1990,end=2017,value=2010)
    def update_map(event):
        World_Shape_Data,key = get_dataset(COUNTRY=data_select.value,Year=year_slider.value)        
        map_pane.object = bokeh_plot_map(gdf, key)        
        return
    year_slider.param.watch(update_map,'value')
    year_slider.param.trigger('value')
    data_select.param.watch(update_map,'value')
    app = pn.Column(pn.Row(data_select,year_slider),map_pane)
    return app

app = map_dash()

TypeError: ignored